In [ ]:
# Linear versus quadratic model estimation
# hollander et al., 2021
# test for decoding accuracy
# increase linearly or show a clear peak at a specific cortical depth
# perform bayesian model comparison

# fit two hierarhical linear models using pymc
# NUTS sampler

# dependent variable (decoding accuracy) was first modeled as a linear function of cortical depth
# y_n = theta_n,0 + theta_n,1 * d_n

# and then also as a quadratic function of cortical depth
# y_n = theta_n,0 + theta_n,1 * d_n * theta_n,2 * d^2

# where y_n were all observationf for subjects n
# d is a corresponding vector of cortical depths
# theta_n is the parameter vector for subject n that was estimated

# the subject-wise parameters were modeled as coming from Gaussian group distirbution mean mu and standard deviation sigma
# theta ~ N(mu, sigma)
# mu ~ N(0, 1)
# sigma ~ HalfCauchy(5) -> 5 subjects?

# (1) use the state-of-the-art Watanabe-Akaike informatoin criterion to do Bayesian model copmarison between the linear and quadratic models
# (2) estimate the posterior of the peak of the quadratic function using the formula 

